In [66]:
# import packages
from covid19dh import covid19
from datetime import date
from Get_covid_data import get_data
import pymc3 as pm
import pandas as pd 
import numpy as np 
import seaborn as sns
import arviz as az
import matplotlib.pyplot as plt
import os 
import theano
import random
from sklearn.preprocessing import MinMaxScaler
# import functions from fns file 
import fns as f

Load covid data, create subset/new columns and plot

In [67]:
#Get data
data = get_data(level = 2, start = date(2020,1,1)) #can get more or less data here.
#group_by introduce lag (new infected from commulative)
data["new_infected"] = data.groupby(["administrative_area_level_2"])["confirmed"].diff()
data = data[data["new_infected"].notna()]
##Create subset from 5 states
#subset = data[data['administrative_area_level_2'].isin(["Florida"])]
##Check data
#sns.lineplot(data = data, x = "date", y = "new_infected", hue = "administrative_area_level_2")



We have invested a lot of time and effort in creating COVID-19 Data Hub, please cite the following when using it:

	Guidotti, E., Ardia, D., (2020), "COVID-19 Data Hub", Journal of Open Source Software 5(51):2376, doi: 10.21105/joss.02376.

A BibTeX entry for LaTeX users is

	@Article{,
		title = {COVID-19 Data Hub},
		year = {2020},
		doi = {10.21105/joss.02376},
		author = {Emanuele Guidotti and David Ardia},
		journal = {Journal of Open Source Software},
		volume = {5},
		number = {51},
		pages = {2376},
	}

To hide this message use 'verbose = False'.


Prep data and create train/test split

In [68]:
# specify x, y & grouping variable in orig. data &
# the name that will be used going forward (can be the same). 
y = "new_infected"
x_old = "date"
idx_old = "administrative_area_level_2"
x_new = "date_2"
idx_new = "idx"

# get idx for each group (e.g. country) and zero-index for time variable (e.g. year).
d = f.get_idx(data, idx_old, x_old, idx_new, x_new)

# use test/train split function
train, test = f.train_test(d, x_new) # check doc-string.

#Scale values:
scaler = MinMaxScaler()
train['new_infected'] = scaler.fit_transform(train['new_infected'].values.reshape(-1,1))
test['new_infected'] = scaler.fit_transform(test['new_infected'].values.reshape(-1,1))

# we need N here as well for shape.
N = len(np.unique(train[idx_new]))

<ipython-input-68-e560b23adba9>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['new_infected'] = scaler.fit_transform(train['new_infected'].values.reshape(-1,1))


<ipython-input-65-1baac83fd9ff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Col1_scaled'] = scaler.fit_transform(train['new_infected'].values.reshape(-1,1))


Build model

In [70]:
# pooled model
with pm.Model() as m1: 
    
    # hyper-priors
    α_μ = pm.Normal('α_μ', mu=0, sd=1)
    α_σ = pm.HalfNormal('α_σ', 1)
    β_μ = pm.Normal('β_μ', mu=0, sd=1)
    β_σ = pm.HalfNormal('β_σ', sd=1)
    
    # priors
    α = pm.Normal('α', mu=α_μ, sd=α_σ, shape=N)
    β = pm.Normal('β', mu=β_μ, sd=β_σ, shape=N)
    ε = pm.HalfCauchy('ε', 1)
    
    # containers 
    x_ = pm.Data('m1_x', train[x_new].values) #Year_ as data container (can be changed). 
    idx_ = pm.Data('m1_idx', train[idx_new].values)
    
    y_pred = pm.Normal('y_pred',
                       mu=α[idx_] + β[idx_] * x_,
                       sd=ε, observed=train[y].values)
    
    # trace 
    m1_trace = pm.sample(2000, return_inferencedata = True,
                        target_accept = .99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [ε, β, α, β_σ, β_μ, α_σ, α_μ]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 2842 seconds.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


In [ ]:
# check trace
az.plot_trace(m1_trace)

In [ ]:
# posterior predictive on new data. 
predictions = f.pp_test(
    m1, 
    m1_trace, 
    {'m1_x': test[x_new].values, 'm1_idx': test[idx_new].values},
    params = ["α", "β", "y_pred"]
    )

In [ ]:
# plot posterior predictive on new data. 
f.plot_pp(predictions, train, test, d, x_new, y, idx_old)